### Connection With Azure Lake ###

In [0]:
dbutils.secrets.list(scope='ali-scope')

[SecretMetadata(key='appsecret'),
 SecretMetadata(key='sec-id'),
 SecretMetadata(key='ten-id')]

In [0]:


spark.conf.set("fs.azure.account.auth.type.datatricksexternal190.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.datatricksexternal190.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.datatricksexternal190.dfs.core.windows.net", dbutils.secrets.get(scope='ali-scope',key='appsecret'))
spark.conf.set("fs.azure.account.oauth2.client.secret.datatricksexternal190.dfs.core.windows.net",dbutils.secrets.get(scope='ali-scope',key='sec-id'))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.datatricksexternal190.dfs.core.windows.net", "https://login.microsoftonline.com/ce69d97e-12e0-4b6d-8f1a-0c5d7534dae8/oauth2/token")

### Fetching data

In [0]:
dbutils.fs.ls('/Volumes/databrickstut/default/ps')

[FileInfo(path='dbfs:/Volumes/databrickstut/default/ps/Features data set.csv', name='Features data set.csv', size=600478, modificationTime=1755821335000),
 FileInfo(path='dbfs:/Volumes/databrickstut/default/ps/sales data-set.csv.zip', name='sales data-set.csv.zip', size=3073157, modificationTime=1755821338000),
 FileInfo(path='dbfs:/Volumes/databrickstut/default/ps/stores data-set.csv', name='stores data-set.csv', size=577, modificationTime=1755821334000)]

In [0]:
import zipfile
import os

with zipfile.ZipFile('/Volumes/databrickstut/default/ps/sales data-set.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/Volumes/databrickstut/default/ps')


df_sales = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema", "true")\
                .load('/Volumes/databrickstut/default/ps/sales data-set.csv')  

df_features = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema", "true")\
                .load('/Volumes/databrickstut/default/ps/Features data set.csv')          

df_stores = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema", "true")\
                .load('/Volumes/databrickstut/default/ps/stores data-set.csv')                                 

In [0]:
import requests
import pandas as pd
import json



url_holidays='https://date.nager.at/api/v3/publicholidays/2025/AT'
response=requests.get(url_holidays)
h=response.json()
df=pd.DataFrame(h)
df_holidays=spark.createDataFrame(df)

url_weather='https://api.open-meteo.com/v1/forecast?latitude=40.7128&longitude=-74.0060&hourly=temperature_2m,precipitation&current_weather=true'


responsew=requests.get(url_weather)
w=responsew.json()
df=pd.DataFrame(w['hourly'])
df_weather=spark.createDataFrame(df)



                            

### Pushing To Lake As Bronze Layer

In [0]:
df_sales.write.format("parquet")\
    .mode("overwrite")\
    .option(
        'path','abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/retail/sales')\
    .save()



df_features.write.format("parquet")\
    .mode("overwrite")\
    .option(
        'path','abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/retail/features')\
    .save()    


df_stores.write.format("parquet")\
    .mode("overwrite")\
    .option(
        'path','abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/retail/stores')\
    .save()    




---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-8550847607893737>, line 26
      9 df_features.write.format("parquet")\
     10     .mode("overwrite")\
     11     .option(
     12         'path','abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/retail/features')\
     13     .save()    
     16 df_stores.write.format("parquet")\
     17     .mode("overwrite")\
     18     .option(
     19         'path','abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/retail/stores')\
     20     .save()    
     23 df_holidays.write.format("parquet")\
     24     .mode("overwrite")\
     25     .option("path", 'abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/holidays/')\
---> 26     .save()

File /databricks/spark/python/pyspark/sql/connect/readwriter.py:670, in DataFrameWriter.save(self, path, format, mode, partit

In [0]:
df_holidays.display()

date,localName,name,countryCode,fixed,global,counties,types
2025-01-01,Neujahr,New Year's Day,AT,false,true,null,[Public]
2025-01-06,Heilige Drei Könige,Epiphany,AT,false,true,null,[Public]
2025-04-20,Ostersonntag,Easter Sunday,AT,false,true,null,[Public]
2025-04-21,Ostermontag,Easter Monday,AT,false,true,null,[Public]
2025-05-01,Staatsfeiertag,National Holiday,AT,false,true,null,[Public]
2025-05-29,Christi Himmelfahrt,Ascension Day,AT,false,true,null,[Public]
2025-06-08,Pfingstsonntag,Pentecost,AT,false,true,null,[Public]
2025-06-09,Pfingstmontag,Whit Monday,AT,false,true,null,[Public]
2025-06-19,Fronleichnam,Corpus Christi,AT,false,true,null,[Public]
2025-08-15,Maria Himmelfahrt,Assumption Day,AT,false,true,null,[Public]


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df_holidays=df_holidays.drop('launchYear','counties')
df_holidays=df_holidays.withColumn('types', col('types').cast(StringType()))



In [0]:
df_weather.write.format("parquet")\
    .mode("overwrite")\
    .option('path', 'abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/weather/')\
    .save()    

In [0]:
df_holidays.write.format("parquet")\
    .mode("overwrite")\
    .option('path', 'abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Bronze/holidays/')\
    .save()    